In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%%capture --no-stderr
!pip install python-dotenv openAI langchain_core langchain_openai

In [ ]:
# 환경변수 설정

In [ ]:
# 라이브러리 불러오기
from dotenv import load_dotenv
import os
from langchain_openai import OpenAI

In [ ]:
# .env 파일에서 환경 변수 로드
load_dotenv("/content/.env")
# 환경 변수에서 API 키 가져오기
api_key = os.getenv("OPENAI_API_KEY")
# 오픈AI 대규모 언어 모델 초기화
llm = OpenAI()

# **호출(invoke) 출력**

In [ ]:
# 라이브러리 설치
!pip install langchain_core langchain_openai
# 라이브러리 불러오기
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 오픈AI의 대규모 언어 모델 설정
model = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
# 프롬프트 템플릿 정의: 주어진 주제에 대한 설명 요청
prompt = ChatPromptTemplate.from_template("주제 {topic}에 대해 짧은 설명을 해주세요.")
# 출력 파서 정의: AI 메시지의 출력 내용을 추출
parser = StrOutputParser()
# 프롬프트, 모델, 출력 파서를 체인으로 연결
chain = prompt | model | parser

In [ ]:
# 응답 호출
chain.invoke({"topic": "더블딥"})

# **배치(Batch) 출력**

In [ ]:
# 주어진 주제 리스트를 배치로 출력
chain.batch([{"topic": "더블딥"}, {"topic": "인플레이션"}])

# **스트림(Stream) 출력**

In [ ]:
# 응답을 토큰 단위로 스트리밍하여 출력
for token in chain.stream({"topic": "더블딥"}):
  # 스트리밍된 내용을 출력, 각 내용을 붙여서 출력하며 버퍼를 즉시 플러시하여 실시간으로 보여줌
  print(token, end="", flush=True)

# **구성된 체인을 다른 러너블과 결합하기**

In [ ]:
# 라이브러리 불러오기
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parser import StrOutputParser

# 이 대답을 영어로 번역해 주세요 라는 질문을 생성하는 프롬프트 템플릿 정의
analysis_prompt = ChatPromptTemplate.from_template("이 대답을 영어로 번역해 주세요: {answer}")

In [ ]:
# 이전에 정의된 체인과 새로운 작업을 연결하는 체인 구성
composed_chain = {"answer": chain} | analysis_prompt | model | StrOutputParser()

# 더블딥이라는 주제로 응답을 생성하고 체인 실행
composed_chain.invoke({"topic": "더블딥"})

# **람다 함수를 사용한 체인을 통해 구성하기**

In [ ]:
# 이전에 정의된 값들
model = ChatOpenAI(model="gpt-4o-mini")
prompt = ChatPromptTemplate.from_template("{topic}에 대해 짧은 설명을 해주세요.")
chain = prompt | model | StrOutputParser()

analysis_prompt = ChatPromptTemplate.from_template("이 대답을 영어로 번역해 주세요: {answer}")

# 람다 함수를 사용한 체인 구성
composed_chain_with_lambda = (
    # 이전에 정의된 체인을 사용하여 입력된 데이터를 받아온다
    chain
    # 입력된 데이터를 answer 키로 변환하는 람다 함수를 적용한다
    # 람다 함수 = 익명 함수
    # 매개변수 : 함수 내용
    # 즉 딕셔너리 반환
    | (lambda input: {"answer": input})
    # answer 키를 가진 데이터를 영어로 번역하도록 프롬프트에 전달한다
    | analysis_prompt
    # 프롬프트에서 생성된 요청을 모델에 전달하여 결과를 생성한다
    | model
    # 모델에서 변환된 결과를 문자열로 파싱한다
    | StrOutputParser()
)

#  더빌딥이라는 주제로 답변을 생성하고 답변을 영어로 번역한다
composed_chain_witg_lambda.invoke({"topic": "더블딥"})

# **.pipe()를 통해 체인 구성하기**

In [ ]:
# 방법1 여러 작업을 순차적으로 pipe를 통해 연결하여 체인 구성하기
composed_chain_with_pipe = (
    # 이전에 정의된 체인으로 입력된 데이터를 받아옴
    chain
    # 입력된 데이터를 answer 키로 변환하는 람다 함수 적용
    .pipe(lambda input: {"answer": input})
    # analysis_prompt를 체인에 연결하여 설명을 영어로 번역하는 작업 추가
    .pipe(analysis_prompt)
    # 모델을 사용해 응답 생성
    .pipe(model)
    # 생성된 응답을 문자열로 파싱
    .pipe(StrOutputParser())
)

# 더블딥이라는 주제로 체인을 실행하여 답변 생성
composed_chain_with_pipe.invoke({"topic": "더블딥"})

In [ ]:
# 방법2 좀 더 간단하게 연결하기
composed_chain_with_pipe = chain.pipe(lambda input: {"answer": input}, analysis_prompt, model, StrOutputParser())

# 더블딥이라는 주제로 체인을 실행하여 답변 생성
composed_chain_with_pipe.invoke({"topic": "더블딥"})